<a href="https://colab.research.google.com/github/siggu/Python/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D_9_%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%94%84%EB%A0%88%EC%9E%84_%ED%95%B5%EC%8B%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9. 데이터프레임 핵심

In [1]:
import pandas as pd
import numpy as np

np.random.seed(12345)
np.set_printoptions(precision=4, suppress=True)

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

PREVIOUS_MAX_ROWS = pd.options.display.max_rows # 원래 60이 기본.
pd.set_option("max_rows", 20)

# 9.1. 리인덱싱

## 9.1.1. 시리즈 리인덱싱

In [2]:
obj = pd.Series([4.5, 7.2, -5.3, 4.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    4.6
dtype: float64

새로운 인덱스가 추가되면 `NaN`이 사용된다.

In [3]:
obj2 = obj.reindex(['a', 'b', 'c,' 'd', 'e'])
obj2

a     -5.3
b      7.2
c,d    NaN
e      NaN
dtype: float64

지정되지 않은 인덱스는 무시된다.

In [4]:
obj2 = obj.reindex(['a', 'c', 'd', 'e'])
obj2

a   -5.3
c    4.6
d    4.5
e    NaN
dtype: float64

## 9.1.2. 데이터프레임 리인덱싱

데이터프레임은 (행의) `index`와 (열의) `columns` 속성에 대해 리인덱싱이 가능하며 작동법은 시리즈의 인덱싱과 동일하다.

In [5]:
frame = pd.DataFrame(np.arange(9).reshape((3,3)),
                     index=['a', 'c', 'd'],
                     columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


`reindex()` 메서드는 기본적으로 행의 `index`에 대해 작동한다.

In [6]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


열의 `columns`에 대해서는 `colunms` 키워드 인자를 활용한다.

In [7]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


`reindex()` 메서드와 함께 사용할 수 있는 키워드 인자가 더 있지만 여기서는 다루지 않는다.

# 9.2. 결측치 처리

**결측치 채우기 1** : `method` 키워드 인자

리인덱싱 과정에서 결측치가 발생할 때 여러 방식으로 채울 수 있다. `method='fill'` 키워드 인자는 결측치를 위쪽에 위치한 값으로 채운다.

**주의사항** : 인덱스가 오름 또는 내림 차순으로 정렬되어 있는 경우에만 가능하다.

In [8]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 5])
obj3

0      blue
2    purple
5    yellow
dtype: object

In [10]:
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    purple
5    yellow
dtype: object

물론 위쪽에 위치한 값이 없으면 결측치가 된다.

In [11]:
obj3.reindex(range(-1, 6), method='ffill')

-1       NaN
0       blue
1       blue
2     purple
3     purple
4     purple
5     yellow
dtype: object

아래쪽에 있는 값으로 채울 수도 있다.

In [12]:
obj3.reindex(range(-1,6), method='bfill')

-1      blue
0       blue
1     purple
2     purple
3     yellow
4     yellow
5     yellow
dtype: object

아니면 가장 가까운 곳에 있는 값으로 채울 수도 있다. 1번 인덱스의 경우처럼 거리가 같으면 아래쪽에서 택한다.

In [14]:
obj3.reindex(range(-1, 6), method='nearest')

-1      blue
0       blue
1     purple
2     purple
3     purple
4     yellow
5     yellow
dtype: object

**결측치 채우기 2** : `fill_value` 키워드 인자

리인덱싱 과정에서 발생하는 모든 결측치를 지정된 값으로 대체할 수 있다. 기본값은 `NaN`이다.

In [15]:
obj3.reindex(range(-1, 6), fill_value='No Color')

-1    No Color
0         blue
1     No Color
2       purple
3     No Color
4     No Color
5       yellow
dtype: object

리인덱싱은 항상 새로운 시리즈를 생성한다. 따라서 `obj3` 자체는 변하지 않는다.

# 9.3. 인덱싱, 슬라이싱, 필터링

## 9.3.1. 시리즈의 인덱싱, 슬라이싱, 필터링

시리즈의 경우 1차원 넘파이 어레이와 거의 동일하게 작동한다. 다만 정수 대신에 지정된 인덱스를 사용할 때 조금 차이가 있다.

In [17]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [18]:
obj['b']

1.0

In [19]:
obj[1]

1.0

In [20]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

여러 개의 인덱스를 리스트로 지정하여 인덱싱을 진행할 수 있다.

In [22]:
obj[['b', 'a', 'd']]

b    1.0
a    0.0
d    3.0
dtype: float64

In [23]:
obj[[1,3]]

b    1.0
d    3.0
dtype: float64

필터링(부울 인덱싱)은 동일하게 작동한다.

In [25]:
obj < 2

a     True
b     True
c    False
d    False
dtype: bool

In [26]:
obj[obj<2]

a    0.0
b    1.0
dtype: float64

정수가 아닌 다른 인덱스 이름, 즉 라벨을 이용하는 슬라이싱은 양쪽 구간의 끝을 모두 포함하는 점이 다르다.

In [27]:
obj['b':'c']

b    1.0
c    2.0
dtype: float64

In [28]:
obj['b':'c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

**주의사항** : 라벨 슬라이싱은 기본적으로 알파벳 순서를 따르며 시리즈에 사용된 순서와 상관 없다.

In [30]:
obj.reindex(['b', 'd', 'c', 'a'])

b    5.0
d    3.0
c    5.0
a    0.0
dtype: float64

In [31]:
obj['b':'d']

b    5.0
c    5.0
d    3.0
dtype: float64

## 9.3.2. 데이터프레임의 인덱싱, 슬라이싱, 필터링

2차원 넘파이 어레이와 거의 유사하게 작동한다.

In [32]:
data = pd.DataFrame(np.arange(16).reshape((4,4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


인덱싱은 기본적으로 열을 기준으로 진행된다.

In [33]:
data['two']

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int64

In [34]:
data[['three', 'one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


하지만 숫자 슬라이싱은 행을 기준으로 작동한다.

In [36]:
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


필터링(부울 인덱싱) 또한 넘파이 2차원 어레이와 동일하게 작동한다.

In [37]:
mask1 = data['three'] > 5
mask1

Ohio        False
Colorado     True
Utah         True
New York     True
Name: three, dtype: bool

In [38]:
data[mask1]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


필터링을 이용하여 특정 항목의 값을 업데이트할 수도 있다.

- `~mask1`은 `mask1`의 부정을 나타낸다.

In [39]:
data[~mask1] = 0
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


각각의 항목에 대한 필터링도 비슷하게 작동한다.

In [40]:
mask2 = data < 6
mask2

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,True,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [41]:
data[mask2] = 0
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,0,6,7
Utah,8,9,10,11
New York,12,13,14,15


**행 단위 인덱싱/슬라이싱**

`loc()` 또는 `iloc()` 메서드를 사용한다.

- `loc()` 메서드 : 라벨을 이용할 경우
- `iloc()` 메서드 : 정수 인덱스를 이용할 경우

In [43]:
data.loc['Colorado']

one      0
two      0
three    6
four     7
Name: Colorado, dtype: int64

In [44]:
data.iloc[2]

one       8
two       9
three    10
four     11
Name: Utah, dtype: int64

행과 열에 대해 동시에 인덱싱/슬라이싱을 사용할 수 있으며 2차원 넘파이 어레이가 작동하는 방식과 비슷하다.

In [46]:
data.loc['Colorado', ['two', 'three']]

two      0
three    6
Name: Colorado, dtype: int64

In [48]:
data.iloc[2, [3, 0 ,1]]

four    11
one      8
two      9
Name: Utah, dtype: int64

In [50]:
data.iloc[[1,2],[3, 0, 1]]

,four,one,two
Colorado,7,0,0
Utah,11,8,9


인덱싱/슬라이싱에 이은 필터링을 연달아 적용할 수 있다.

In [51]:
data.iloc[:, :3][data.three > 5]

,one,two,three
Colorado,0,0,6
Utah,8,9,10
New York,12,13,14


## 9.3.3. `drop()` 메서드

특정 행 또는 열의 인덱스를 제외한 나머지로 이루어진 시리즈/데이터프레임을 구할 때 사용한다.

시리즈의 경우 인덱스를 한 개 또는 여러 개 지정하면 나머지로 이루어진 시리즈를 얻는다.

In [54]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [55]:
new_obj = obj.drop('c')
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [56]:
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

원래의 시리즈를 직접 건드리지는 않는다.

In [57]:
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

하지만 `inplace=True` 키워드 인자를 이용하여 원본을 수정할 수도 있다. 물론 사용에 매우 주의해야 한다.

In [58]:
obj.drop('c', inplace=True)

In [59]:
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

데이터프레임의 경우도 기본적으로 행의 인덱스를 기준으로 작동한다.

In [63]:
data = pd.DataFrame(np.arange(16).reshape((4,4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [64]:
data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


열을 기준으로 작동하게 하려면 `axis=1`로 지정한다.

In [65]:
data.drop('two', axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


`axis='columns'`로 지정해도 된다.

In [66]:
data.drop(['two', 'four'], axis='columns')

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


`inplace=True` 키워드 인자를 사용하면 원본을 수정한다.

In [68]:
data.drop(['two', 'four'], axis='columns', inplace=True)

data

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


# 9.4. 산술 연산

시리즈/데이터프레임의 사직 연산은 기본적으로 아래 원칙을 따른다.

- 연산에 사용된 모든 인덱스는 포함
- 공통으로 사용되는 인덱스의 항목에 대해서만 연산 적용. 그렇지 않으면 `NaN`으로 처리.

In [72]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [74]:
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1], index=['a', 'c', 'e', 'f', 'g'])

s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [76]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

In [78]:
df1 = pd.DataFrame(np.arange(9.).reshape((3,3)),
                   columns=list('bcd'),
                   index=['Ohio', 'Texas', 'Colorado'])
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [82]:
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                   columns=list('bce'),
                   index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df2

,b,c,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [83]:
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,5.0,NaN,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,11.0,NaN,NaN
Utah,NaN,NaN,NaN,NaN


기본적으로 사용되는 산술 연산 기호에 해당하는 메서드가 존재한다. 아래는 가장 많이 상요되는 연산 메서드들이다.

|메서드|설명|
|:---:|:---:|
|`add()`|덧셈 계산 메서드|
|`sub()`|뺄셈 계산 메서드|
|`mul()`|곱셈 계산 메서드|
|`div()`|나눗셈 계산 메서드|
|`floordiv()`|몫 계산 메서드|
|`pow()`|거듭제곱 메서드|

## 9.4.1. 연산과 결측치

공통 인덱스가 아니거나 결측치가 이미 존재하는 경우 기본적으로 결측치로 처리된다. 하지만 `fill_value` 키워드 인자를 이용하여 지정된 값으로 처리하게 만들 수도 있다. 다만, 연산 기호 대신에 해당 연산의 메서드를 활용해야 한다.

In [84]:
df1.add(df2, fill_value=0)

,b,c,d,e
Colorado,6.0,7.0,8.0,NaN
Ohio,3.0,5.0,2.0,5.0
Oregon,9.0,10.0,NaN,11.0
Texas,9.0,11.0,5.0,8.0
Utah,0.0,1.0,NaN,2.0


## 9.4.2. 브로드캐스팅

넘파이에서 2차원 어레이와 1차원 어레이 사이에 브로드캐스팅이 가능한 경우, 즉, 차원을 맞출 수 있는 경우에 연산이 가능했다.

In [85]:
arr = np.arange(12.).reshape((3,4))
arr

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]])

In [86]:
arr[0]

array([0., 1., 2., 3.])

브로드캐스팅이 불가능하면 오류가 발생한다.

In [88]:
arr[:,1]

array([1., 5., 9.])

In [89]:
try:
  arr + arr[:,1]
except:
  print("브로드캐스팅 불가능")

브로드캐스팅 불가능


물론 아래와 같이 브로드캐스팅이 가능하도록 모양을 변환한 다음에는 연산이 가능하다.

In [90]:
arr_1 = arr[:,1][:, np.newaxis]
arr_1

array([[1.],
       [5.],
       [9.]])

In [91]:
arr + arr_1

array([[ 1.,  2.,  3.,  4.],
       [ 9., 10., 11., 12.],
       [17., 18., 19., 20.]])

데이터프레임과 시리즈 사이의 연산도 동일하게 작동한다. 다만, 행 또는 열에 대한 연산 여부를 확실하게 구분해주어야 한다.

In [92]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                     columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])

frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [96]:
series=frame.iloc[0]
series

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

브로드캐스팅은 기본적으로 행 단위로 이루어진다. 따라서 아래처럼 데이터프레임과 시리즈의 연산을 그냥 적용할 수 있다.

In [97]:
frame - series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


공통 인덱스가 존재하면 두 인자 모두에 대해 브로드캐스팅이 적용된다.

In [99]:
series2 = pd.Series(range(3), index=['b', 'e', 'f'])
series2

b    0
e    1
f    2
dtype: int64

In [101]:
frame + series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


열 단위로 데이터프레임과 시리즈를 더하려면 해당 연산 메서드를 `axis=0` 키워드 인자와 함께 적용해야 한다.

In [104]:
series3 = frame['d']
series3

Utah       1.0
Ohio       4.0
Texas      7.0
Oregon    10.0
Name: d, dtype: float64

In [105]:
frame.sub(series3, axis=0)

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


`axis='index'`를 사용해도 된다.

In [106]:
frame.sub(series3, axis='index')

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


## 9.4.3. 유니버설 함수

유니버설 함수는 넘파이의 경우와 동일하게 작동한다.

In [108]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0.274992,0.228913,1.352917
Ohio,0.886429,-2.001637,-0.371843
Texas,1.669025,-0.438570,-0.539741
Oregon,0.476985,3.248944,-1.021228


넘파이의 `abs()` 함수를 적용하면 항목별로 이루어진다.

In [109]:
np.abs(frame)

,b,d,e
Utah,0.274992,0.228913,1.352917
Ohio,0.886429,2.001637,0.371843
Texas,1.669025,0.438570,0.539741
Oregon,0.476985,3.248944,1.021228


시리즈에 대해서도 동일하다.

In [110]:
np.abs(frame['b'])

Utah      0.274992
Ohio      0.886429
Texas     1.669025
Oregon    0.476985
Name: b, dtype: float64

`map()`과 `applymap()` 메서드

유니버설 함수가 아닌 함수를 시리즈의 항목별로 적용하려면 `map()` 메서드를 이용한다.

예를 들어 아래 람다(lambda) 함수는 부동소수점을 소수점 이하 셋째 자리에서 반올림한 값만 보여주도록 한다.

In [111]:
format = lambda x: '%.2f' % x

시리즈에 적용해보자.

In [112]:
frame['e'].map(format)

Utah       1.35
Ohio      -0.37
Texas     -0.54
Oregon    -1.02
Name: e, dtype: object

유니버설 함수가 아닌 함수를 데이터프레임의 항목별로 적용하려면 `applymap()` 메서드를 이용한다.

In [113]:
frame.applymap(format)

,b,d,e
Utah,0.27,0.23,1.35
Ohio,0.89,-2.00,-0.37
Texas,1.67,-0.44,-0.54
Oregon,0.48,3.25,-1.02


`apply()` 메서드

행 또는 열 단위로 함수를 적용하려면 `apply()` 메서드를 활용한다. 기본은 열 단위로 함수가 적용되며 반환값이 스칼라 값이면 시리즈가 반환된다.

예를 들어 아래 함수는 최댓값과 최소값의 차이를 반환한다.

In [115]:
f1 = lambda x: x.max() - x.min()

데이터프레임에 적용하면 열 별로 최댓값과 최솟값의 차이를 계산하여 시리즈로 반환한다.

In [116]:
frame.apply(f1)

b    1.394034
d    5.250581
e    2.374144
dtype: float64

행 별로 함수를 적용하려면 `axis=1` 또는 `axis='columns'`를 지정해야 한다.

In [119]:
frame.apply(f1, axis='columns')

Utah      1.124004
Ohio      2.888067
Texas     2.208767
Oregon    4.270171
dtype: float64

In [120]:
frame.apply(f1, axis=1)

Utah      1.124004
Ohio      2.888067
Texas     2.208767
Oregon    4.270171
dtype: float64

함수의 최댓값이 시리즈이면 `apply()` 메서드는 데이터프레임을 반환한다. 예를 들어 아래 함수는 최솟값과 최댓값을 갖는 시리즈를 반환한다.

In [121]:
def f2(x):
  return pd.Series([x.min(), x.max()], index=['min','max'])

`apply()` 메서드와 함께 호출하면 열 별로 최댓값과 최솟값을 계산하여 데이터프레임으로 반환한다.

In [122]:
frame.apply(f2)

,b,d,e
min,0.274992,-2.001637,-1.021228
max,1.669025,3.248944,1.352917


**참고** : 시리즈 객체 또한 `apply()` 메서드를 갖는다. 하지만 기본적으로 `map()` 메서드처럼 작동한다. `map()` 메서드보다 좀 더 다양한 기능을 갖지만 여기서는 다루지 않는다.

# 9.5. 정렬

행과 열의 인덱스 또는 항목을 대상으로 정렬할 수 있다.

`sort_index()` 메서드

시리즈의 경우 인덱스를 기준으로 정렬한다.

In [124]:
obj = pd.Series(range(4), index=['d', 'a', 'b', 'c'])
obj

d    0
a    1
b    2
c    3
dtype: int64

In [125]:
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

내림차순으로 정렬하려면 `ascending=False` 키워드 인자를 함께 사용한다.

In [126]:
obj.sort_index(ascending=True)

a    1
b    2
c    3
d    0
dtype: int64

데이터프레임의 경우 행 또는 열의 인덱스를 기준으로 정렬한다.

In [127]:
frame = pd.DataFrame(np.arange(8).reshape((2,4)),
                     index=['three', 'one'],
                     columns=['d', 'a', 'b', 'c'])
frame

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


기본은 행의 인덱스를 기준으로 정렬한다.

In [128]:
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


열의 인덱스를 기준으로 정렬하려면 `axis=1` 또는 `axis='columns'` 키워드 인자를 사용한다.

In [130]:
frame.sort_index(axis=1)

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


In [131]:
frame.sort_index(axis='columns')

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


내림차순으로 정렬하려면 `asceding=False` 키워드 인자를 함께 사용한다.

In [133]:
frame.sort_index(axis=1, ascending=False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


In [134]:
obj.sort_values()

d    0
a    1
b    2
c    3
dtype: int64

결측치는 맨 나중에 위치시킨다.

In [136]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj

0    4.0
1    NaN
2    7.0
3    NaN
4   -3.0
5    2.0
dtype: float64

In [137]:
obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

데이터프레임의 경우 `by` 키워드 인자를 이용하여 열의 라벨을 지정해야 한다.

In [138]:
frame = pd.DataFrame({'b' : [4, 7, -3, 2], 'a':[0, 1, 0, 1]})
frame

,b,a
0,4,0
1,7,1
2,-3,0
3,2,1


예를 들어, `b` 열의 값을 기준으로 정렬한다. 물론 동일한 행의 값은 함께 움직인다.

In [142]:
frame.sort_values(by='b')

,b,a
2,-3,0
3,2,1
0,4,0
1,7,1


여러 열의 값을 기준으로 정렬하려면 라벨의 리스트를 입력한다. 그러면 리스트 항목 순서대로 기준이 정해진다.

예를 들어 아래 코드는 먼저 `a` 열의 항목들을 순서대로 정렬한 다음에 동등한 값의 경우에는 `b` 열의 항목들 순서대로 정렬한다.

In [143]:
frame.sort_values(by=['a', 'b'])

,b,a
2,-3,0
0,4,0
3,2,1
1,7,1


**참고** : `axis=1`을 이용하여 특정 행의 값을 기준으로 정렬할 수도 있다. 하지만 데이터프레임은 서로 다른 특성을 열 단위로 담는 목적으로 사용되기 때문에 이런 정렬은 사용할 이유가 별로 없다.

In [144]:
frame.sort_values(by=2, axis=1, ascending=False)

,a,b
0,0,4
1,1,7
2,0,-3
3,1,2
